# Hypothesis 2
## The proportion of incarcerated people to the total population differs in correlation with whether a state has R or D senators 

${H_0}: \textrm{There is no difference in the total incarcerated population between states with R and D senators}$
${H_\alpha}: \textrm{There is a difference between the proportion of incarcerated persons to total population between R and D states
}$

$\alpha = 0.05$

### Method

Two-tailed test

In [37]:
import requests
import pandas as pd

In [48]:
# Get Senator information
df = pd.DataFrame()

headers = {'X-API-Key': 'l8xbuedseZHG4JrxvS9u40eyNMOWRTVDL4IfnIb6'}
cong_nums = ['112', '113', '114']
url = 'https://api.propublica.org/congress/v1/112/senate/members.json'
response = requests.get(url, headers=headers)

    
print(response)
print(response.text)

<Response [200]>
{
   "status":"OK",
   "copyright":" Copyright (c) 2019 Pro Publica Inc. All Rights Reserved.",
   "results":[
      {
         "congress": "112",
         "chamber": "Senate",
         
         
         "num_results": 102,
         "offset": 0,
         "members": [
              {
                 "id": "A000069",
                 "title": "Senator, 1st Class",
                 "short_title": "Sen.",
                 "api_uri":"https://api.propublica.org/congress/v1/members/A000069.json",
                 "first_name": "Daniel",
                 "middle_name": "K.",
                 "last_name": "Akaka",
                 "suffix": null,
                 "date_of_birth": "1924-09-11",
                 "gender": "M",
                 "party": "D",
                 "leadership_role": null,
                 "twitter_account": "SenatorAkaka",
                 "facebook_account": null,
                 "youtube_account": "senatorakaka",
                 "govtrack_id": "3

In [57]:
len(response.json()['results'][0]['members'])

102

In [59]:
response.json()['results'][0]['members'][0]['first_name']

'Daniel'

In [60]:
response.json()['results'][0]['members'][0]['last_name']

'Akaka'

In [61]:
response.json()['results'][0]['members'][0]['party']

'D'

In [62]:
response.json()['results'][0]['members'][0]['state']

'HI'

In [106]:
def get_senators():
    cong_nums = ['112', '113', '114']
    df = pd.DataFrame()
    headers = {'X-API-Key': 'l8xbuedseZHG4JrxvS9u40eyNMOWRTVDL4IfnIb6'}
    base_url = 'https://api.propublica.org/congress/v1/{}/senate/members.json'
    
    for cong in cong_nums:
        response = requests.get(base_url.format(cong), headers=headers)
        for i in range(len(response.json()['results'][0]['members'])): 
            tdf = pd.DataFrame([cong, response.json()['results'][0]['members'][i]['first_name'], 
                           response.json()['results'][0]['members'][i]['last_name'],
                           response.json()['results'][0]['members'][i]['party'],
                           response.json()['results'][0]['members'][i]['state']]).T
            df = df.append(tdf)
            
    df.columns = ['congress', 'first_name', 'last_name', 'party', 'state']
    df.reset_index(inplace=True, drop=True)
    return df

In [111]:
senators = get_senators()
senators.head()

,congress,first_name,last_name,party,state
0,112,Daniel,Akaka,D,HI
1,112,Lamar,Alexander,R,TN
2,112,Kelly,Ayotte,R,NH
3,112,John,Barrasso,R,WY
4,112,Max,Baucus,D,MT


In [123]:
state_abbrevs = sorted(senators['state'].unique())

In [126]:
# Load incarceration data
inc_data = pd.read_csv('prison_custody_by_state.csv')
inc_data.head()

,jurisdiction,includes_jails,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Federal,0,"149,852","158,216","168,144","177,600","186,364","190,844","197,285","198,414","205,087","206,968","214,774","216,915","214,989","209,561","195,622","188,311"
1,Alabama,0,"24,741","25,100","27,614","25,635","24,315","24,103","25,253","25,363","27,241","27,345","26,813","26,768","26,825","26,145","25,212","23,745"
2,Alaska,1,"4,570","4,351","4,472","4,534","4,798","5,052","5,151","4,997","5,472","5,369","6,216","6,308","5,081","6,323","5,247","4,378"
3,Arizona,0,"27,710","29,359","31,084","32,384","33,345","35,752","37,700","39,455","40,544","40,130","39,949","40,013","41,031","42,136","42,204","42,248"
4,Arkansas,0,"11,489","11,849","12,068","12,577","12,455","12,854","13,275","13,135","13,338","14,192","14,090","14,043","14,295","15,250","15,784","15,833"


In [135]:
pop_data = pd.read_excel('state_population_estimates.xlsx', sheet_name='Data')
pop_data.head()

,FIPS,State,Month,1970,1971,1972,1973,1974,1975,1976,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,01,AL,1.0,NaN,NaN,3518238,3559590,3.60314e+06,3.65266e+06,3.70698e+06,...,4695523,4738072,4.77176e+06,4792114,4.8063e+06,4820803,4.83385e+06,4.84545e+06,4.8557e+06,4867646
1,01,AL,4.0,3444354,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4779736,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,AL,7.0,NaN,3497076,3539400,3579780,3626499,3678814,3735139,...,4718206,4757938,4785579,4798649,4813946,4827660,4840037,4850858,4860545,4874747
3,02,AK,1.0,NaN,NaN,319987,327504,335803,358616,388570,...,683878,693175,706455,718137,726542,733792,736760,737369,739750,740658
4,02,AK,4.0,302583,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
state_abbrevs

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']